# Quick analysis before neurophys meeting

Before 21/05/25 neurophys meeting

## Multi depth

Find one session with some tuned cells in control

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
project = "colasa_3d-vision_revisions"
session_name = "PZAG16.3c_S20250317"

In [ ]:

# Load neurons df
import flexiznam as flz
import pandas as pd
import numpy as np
from cottage_analysis.analysis import spheres
from cottage_analysis.pipelines import pipeline_utils
from cottage_analysis.analysis.spheres import rf_fitting

flexilims_session = flz.get_flexilims_session(project_id=project)

In [ ]:

neurons_ds = pipeline_utils.create_neurons_ds(
            session_name=session_name,
            flexilims_session=flexilims_session,
            conflicts="skip",
        )

In [ ]:
protocol_base = "SpheresPermTubeReward_multidepth"
# STEP BY STEP. Part 1 sync and load
if True:
    # here we do it step by step for debugging

    flexilims_session = flz.get_flexilims_session(project)
    print("---Start synchronisation...---")
    vs_df_all, trials_df_all = spheres.sync_all_recordings(
        session_name=session_name,
        flexilims_session=flexilims_session,
        project=project,
        filter_datasets={"anatomical_only": 3},
        recording_type="two_photon",
        protocol_base=protocol_base,
        photodiode_protocol=5,
        return_volumes=True,
        conflicts="skip",
    )

    # Add trial number to flexilims
    trial_no_closedloop = len(trials_df_all[trials_df_all["closed_loop"] == 1])
    trial_no_openloop = len(trials_df_all[trials_df_all["closed_loop"] == 0])
    ndepths = len(trials_df_all["depth"].unique())
    flz.update_entity(
        "session",
        name=session_name,
        mode="update",
        attributes={
            "closedloop_trials": trial_no_closedloop,
            "openloop_trials": trial_no_openloop,
            "ndepths": ndepths,
        },
        flexilims_session=flexilims_session,
    )

    suite2p_datasets = flz.get_datasets(
        origin_name=session_name,
        dataset_type="suite2p_rois",
        project_id=project,
        flexilims_session=flexilims_session,
        return_dataseries=False,
        filter_datasets={"anatomical_only": 3},
    )
    suite2p_dataset = suite2p_datasets[0]
    frame_rate = suite2p_dataset.extra_attributes["fs"]

    is_multidepth = "multidepth" in protocol_base


In [ ]:
np.geomspace(2.5, 10240, 13)

In [ ]:
np.geomspace(2.5, 10240, 13)

In [ ]:
target = neurons_ds.path_full.parent
print(target)
import os
[f for f in os.listdir(target) if f.endswith('.npy')]

In [ ]:
coeffs = np.load(target / 'coef_subset.npy')
print(coeffs.shape)

In [ ]:
ndepths=8
frame_shape=(16, 24)
position=[0, 0, 1, 1]
plot_prop=0.99
xlabel="Azimuth (deg)"
ylabel="Elevation (deg)"
fontsize_dict={"title": 15, "label": 10, "tick": 5, "legend":6}
resolution=5
azimuth_limits=(-120, 120)
elevation_limits=(-40, 40)

frame_shape = (16,24)



In [ ]:
print(coeffs.shape[-1])

In [ ]:
import matplotlib.pyplot as plt

for roi in range(10):
    fig=plt.figure()
    fig.suptitle(f"ROI {roi}")

    coef = coeffs[...,roi][:, :-1]
    coef = coef.reshape(coef.shape[0], ndepths, frame_shape[0], frame_shape[1])
    coef_mean = np.mean(coef, axis=0)
    coef_max = np.nanmax(coef_mean)
    plot_x, plot_y, plot_width, plot_height = position

    for i in range(ndepths):
        ax = plt.gcf().add_axes(
            [
                plot_x,
                plot_y - plot_height / ndepths * i,
                plot_width,
                plot_height / ndepths * plot_prop,
            ]
        )
        im = plt.imshow(
            coef_mean[i, :, :],
            origin="lower",
            cmap="bwr",
            extent=[0, 120, -40, 40],
            vmin=-np.round(coef_max, 1),
            vmax=np.round(coef_max, 1),
        )
        if i != ndepths - 1:
            plt.gca().set_xticklabels([])
        if i == ndepths // 2:
            ax.set_ylabel(ylabel, fontsize=fontsize_dict["label"])
        if i == ndepths - 1:
            ax.set_xlabel(xlabel, fontsize=fontsize_dict["label"])
        ax.tick_params(axis="both", labelsize=fontsize_dict["tick"], length=1.5)
        ax.set_xticks([0, 60, 120])

        if i == ndepths - 1:
            ax_pos = ax.get_position()
            ax2 = plt.gcf().add_axes(
                [
                    ax_pos.x1 + ax_pos.width * 0.05,
                    ax_pos.y0,
                    0.005,
                    ax_pos.height / 2,
                ]
            )
            cbar = plt.colorbar(mappable=im, cax=ax2)
            # cbar.set_label("Z-score", fontsize=fontsize_dict["legend"])
            cbar.ax.tick_params(labelsize=fontsize_dict["legend"], length=2, pad=1)
            cbar.set_ticks([-np.round(coef_max, 1), 0, np.round(coef_max, 1)])

In [ ]:
vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)
suite2p_ds = flz.get_datasets_recursively(
    flexilims_session=flexilims_session,
    origin_name=session_name,
    dataset_type="suite2p_traces",
)
fs = list(suite2p_ds.values())[0][-1].extra_attributes["fs"]

rs_arr = np.array([j for i in trials_df_example.RS_stim.values for j in i]) * 100
of_arr = np.degrees([j for i in trials_df_example.OF_stim.values for j in i])

In [ ]:
neurons_ds = pipeline_utils.create_neurons_ds(
        session_name=session_name,
        flexilims_session=flexilims_session,
        conflicts="skip",
    )
neurons_df = pd.read_pickle(neurons_ds.path_full)

In [ ]:

txt = f"{len(neurons_df)} neurons"
good_neurons = neurons_df.query("is_depth_neuron == True")
txt += f", {len(good_neurons)} depth selective"
good_neurons = good_neurons.query("depth_tuning_test_spearmanr_rval_closedloop > 0.4")
txt += f", {len(good_neurons)} spearman R >0.4"
good_neurons = good_neurons.query("depth_tuning_test_spearmanr_pval_closedloop < 0.05")
txt += f", {len(good_neurons)} spearman p <0.05"
good_neurons["max_fit"] = good_neurons.depth_tuning_popt_closedloop_running.map(
    lambda x: np.exp(x[0])
)
good_neurons = good_neurons.query("max_fit < 10")
good_neurons = good_neurons.query("max_fit > 0.5")
txt += f", {len(good_neurons)} with large peak dff"
print(txt)
photodiode_protocol = 5

In [ ]:
# Range of "normal" figure
log_range = {
    "rs_bin_log_min": 0,
    "rs_bin_log_max": 2.5,
    "rs_bin_num": 6,
    "of_bin_log_min": -1.5,
    "of_bin_log_max": 3.5,
    "of_bin_num": 11,
    "log_base": 10,
}
rs_bins = (
    np.logspace(
        log_range["rs_bin_log_min"],
        log_range["rs_bin_log_max"],
        num=log_range["rs_bin_num"],
        base=log_range["log_base"],
    )
    # / 100
)
rs_bins = np.insert(rs_bins, 0, 0)

of_bins = np.logspace(
    log_range["of_bin_log_min"],
    log_range["of_bin_log_max"],
    num=log_range["of_bin_num"],
    base=log_range["log_base"],
)
of_bins = np.insert(of_bins, 0, 0)


def rs2pos(x):
    return np.log10(x + 1) / np.log10(rs_bins.max() + 1) * (len(rs_bins)) - 1


def of2pos(x):
    rng = np.log10(of_bins.max() / of_bins[1])
    return np.log10(x / of_bins[1]) / (rng) * (len(of_bins) - 1)

In [ ]:
import scipy
roi = good_neurons.iloc[6].name
ax0 = plt.subplot(1,1,1)
dff_arr = np.vstack(trials_df_example.dff_stim.values)[:, roi]
cmap='Reds'
bin_means, rs_edges, of_edges, _ = scipy.stats.binned_statistic_2d(
    x=rs_arr,
    y=of_arr,
    values=dff_arr,
    statistic="mean",
    bins=[rs_bins, of_bins],
)
vmax = max(np.nanmax(bin_means[1:, 1:]), 0.1)
im = ax0.imshow(
    bin_means[1:, 1:].T,
    origin="lower",
    aspect="equal",
    cmap=cmap,
    vmin=0,
    vmax=vmax,
    extent=np.log10(
        (rs_edges[1], rs_edges[-1], of_edges[1], of_edges[-1])
    ),
)

In [ ]:

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
matplotlib.rcParams['svg.fonttype'] = 'none' # for svgs
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from cottage_analysis.summary_analysis import depth_responses, summary_utils, get_session_list, depth_decoder_stats
from cottage_analysis.plotting import depth_selectivity_plots, depth_decoder_plots, plotting_utils

In [ ]:
depth_tuning_kwargs = dict(
    rs_thr=None,
    plot_fit=True,
    plot_smooth=False,
    linewidth=1.5,
    linecolor="royalblue",
    closed_loop=1,
    fontsize_dict=fontsize_dict,
    markersize=8,
    markeredgecolor='w',
)
roi_num=1
ax = plt.subplot(1,1,1)
depth_selectivity_plots.plot_running_stationary_depth_tuning(roi=roi, roi_num=roi_num, i=roi, 
                                                           neurons_df=neurons_df,
                                                           trials_df=trials_df_example,
                                                           ax=ax, fov_ax=fov_ax, ops=ops, stat=stat,
                                        depth_tuning_kwargs=depth_tuning_kwargs, 
                                        fontsize_dict=fontsize_dict, legend_loc="upper right", text_pos=label_pos)